In [1]:
##vs branch testing

#what to do for visualizzation piece in notebooks

##vidsuaslizations fine in JN

##databricks

#nlp visualizations 
## we can do a heatmap of words or phrases that are "spammy"
##we can do a word cloud of words or phrases that are "spammy"

#unsupervised learning

In [2]:
#lets get crackn
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer #vectoring is after preporcessing
from sklearn.model_selection import train_test_split
#download stuff for nltk if need be
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vincehsanchez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
##^^i think 'True' is a good thing..

In [9]:
#lets build an oven (corpus)
##ham_folder_easy_uno = 'Resources/easy_ham_uno'
##spam_folder_uno = 'Resources/spam_uno'
ham_test_folder = 'Resources/ham_testing_renaming'
#^^paths to our email datasets^^
#we need to mark which emails are spam or ham so have lists for emails and labels
emails = []
labels = []
#read the text files from our folders and give them labels
def read_files(folder, label):
    files = os.listdir(folder)
    for file in files:
        with open(os.path.join(folder, file), 'r', encoding='Windows-1252') as f:
            #UTF-8 does not work, Windows-1252 works!
            ##use chardet if we have any hiccups...
            #https://stackoverflow.com/questions/436220/how-to-determine-the-encoding-of-text
            emails.append(f.read())
            labels.append(label)
#read ham and spam
read_files(ham_test_folder, 'test')
##read_files(ham_folder_easy_uno, 'ham')
##read_files(spam_folder_uno, 'spam')

In [10]:
#lets see if the oven cooks everything evenly
#test if our list is properly read and we see the same amount of emails as we asked it to read
ham_test_df = pd.DataFrame({'Email':emails, 'Label':labels})
#save as csv
ham_test_df.to_csv('Resources/Labeled_CSVs/ham_testing_labeled.csv', index=False)

NameError: name 'df' is not defined